סינון

In [ ]:
# הגדר את שם משתנה המטרה
target_variable = 'dead_5y'


# פיצול המשתנים הנומריים והקטגוריאליים
numeric_columns = data_df.select_dtypes(include=['float64', 'int64']).columns
categorical_columns = data_df.select_dtypes(include=['object', 'category']).columns


# הסרת משתנה המטרה מהרשימות
numeric_columns = [col for col in numeric_columns if col != target_variable]
categorical_columns = [col for col in categorical_columns if col != target_variable]


# משתנה מטרה נומרי - חישוב קורלציה
if data_df[target_variable].dtype in ['float64', 'int64']:
    print("Correlation with Target Variable:")
    for col in numeric_columns:
        correlation, p_value = stats.pearsonr(data_df[col], data_df[target_variable])
        print(f"{col}: Correlation={correlation:.3f}, p-value={p_value:.3f}")


# משתנה מטרה קטגוריאלי - משתנים נומריים
if data_df[target_variable].dtype in ['object', 'category', 'int64']:
    print("\nNumerical Features vs Categorical Target Variable:")
    for col in numeric_columns:
        # בדיקת נורמליות
        groups = [group[col].dropna() for name, group in data_df.groupby(target_variable)]
        is_normal = all(stats.shapiro(group).pvalue > 0.05 for group in groups if len(group) > 3)
        if is_normal:
            t_stat, p_value = stats.f_oneway(*groups)  # מבחן ANOVA
            test_type = "ANOVA"
        else:
            t_stat, p_value = stats.kruskal(*groups)  # מבחן Kruskal-Wallis
            test_type = "Kruskal-Wallis"
        print(f"{col}: Test={test_type}, Statistic={t_stat:.3f}, p-value={p_value:.3f}")


# משתנה מטרה נומרי - משתנים קטגוריאליים
if data_df[target_variable].dtype in ['float64', 'int64']:
    print("\nCategorical Features vs Numerical Target Variable:")
    for col in categorical_columns:
        groups = [group[target_variable].dropna() for name, group in data_df.groupby(col)]
        is_normal = all(stats.shapiro(group).pvalue > 0.05 for group in groups if len(group) > 3)
        if is_normal:
            t_stat, p_value = stats.f_oneway(*groups)  # מבחן ANOVA
            test_type = "ANOVA"
        else:
            t_stat, p_value = stats.kruskal(*groups)  # מבחן Kruskal-Wallis
            test_type = "Kruskal-Wallis"
        print(f"{col}: Test={test_type}, Statistic={t_stat:.3f}, p-value={p_value:.3f}")


# שני משתנים קטגוריאליים - חי בריבוע
if data_df[target_variable].dtype in ['object', 'category', 'int64']:
    print("\nCategorical Features vs Categorical Target Variable:")
    for col in categorical_columns:
        contingency_table = pd.crosstab(data_df[col], data_df[target_variable])
        chi2, p_value, dof, expected = stats.chi2_contingency(contingency_table)
        print(f"{col}: Chi2={chi2:.3f}, p-value={p_value:.3f}, Degrees of Freedom={dof}")
